In [ ]:
# %% Import packages
import datetime
import pandas as pd
import numpy as np

In [ ]:
# %% Define parameters
time_delta = datetime.timedelta(days=14)
start_date = datetime.date(year=2022, month=10, day=15)

In [ ]:
# %% Define functions

def format_date(date_string):
    months_encoded = {"Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04",
                      "May": "05", "Jun": "06", "Jul": "07", "Aug": "08",
                      "Sept": "09", "Oct": "10", "Nov": "11", "Dec": "12"}

    date_list = date_string.split("\n")
    year = int(date_list[1])
    month = int(months_encoded[date_list[0].split(" ")[1]])
    day = int(date_list[0].split(" ")[0])
    return str(datetime.date(year=year, month=month, day=day))

def create_date(date_string):
    date_list = date_string.split("-")
    year = int(date_list[0])
    month = int(date_list[1])
    day = int(date_list[2])
    return (datetime.date(year=year, month=month, day=day))

def calculate_lag(trade, publish):
    trade_date = create_date(trade)
    publish_date = create_date(publish)
    lag = publish_date - trade_date
    return lag.days

def sum_quantity(quant_list):
    sum_value = 0
    for q in quant_list:
        sum_value += int(q)
    return sum_value

def find_unique_titles(title_object):
    unique_titles = []
    for t in title_object:
        if t not in unique_titles:
            unique_titles.append(t)
    return ";".join(unique_titles)

In [ ]:
# %% Load data

imported_all = pd.read_csv("insider_trades.with_titles.csv")
imported_data = imported_all.dropna()

processed_data = {}
processed_data["Insider"] = []
processed_data["Company"] = []
processed_data["Title"] = []
processed_data["Traded"] = []
processed_data["Published"] = []
processed_data["Ticker"] = []
processed_data["last_price"] = []
processed_data["Qty"] = []
processed_data["shares_held"] = []
processed_data["Owned"] = []
processed_data["Value"] = []


for row in imported_data.iterrows():
    # Append the politician data
    processed_data["Insider"].append(row[1]["owner_name"])
    processed_data["Company"].append(row[1]["company_name"])
    processed_data["Title"].append(row[1]["Canonical_Role"])

    # Append the date data
    processed_data["Traded"].append(row[1]["trade_date"])
    processed_data["Published"].append(row[1]["filing_date"].split(" ")[0])

    # Apend the trade information
    processed_data["Ticker"].append(row[1]["ticker"])
    processed_data["last_price"].append(row[1]["last_price"].replace("$", ""))
    processed_data["Qty"].append(row[1]["Qty"].replace("+", "").replace(",", ""))
    processed_data["shares_held"].append(row[1]["shares_held"].replace(",", ""))
    processed_data["Owned"].append(row[1]["Owned"].replace("+", "").replace("%", "").replace(",", ""))
    processed_data["Value"].append(row[1]["Value"].replace("+", "").replace("$", "").replace(",", ""))

processed_data_DF = pd.DataFrame(processed_data)

In [ ]:
# %% Save when sorting by trade date

processed_data_DF_sorted = processed_data_DF.sort_values(by="Traded")
processed_data_DF_sorted.to_csv("insiders_by_trade_date.csv", index=False)

start_temp = start_date
end_temp = start_date + time_delta
end_date = create_date(max([max(processed_data_DF_sorted["Traded"]), max(processed_data_DF_sorted["Published"])]))

biweekly_data = {}
biweekly_data["start_date"] = []
biweekly_data["end_date"] = []
biweekly_data["ticker"] = []
biweekly_data["lag"] = []
biweekly_data["n_trades"] = []
biweekly_data["Qty"] = []
biweekly_data["Title"] = []

while end_temp < end_date:

    temp1 = processed_data_DF_sorted[processed_data_DF_sorted["Traded"] > str(start_temp)]
    temp2 = temp1[temp1["Traded"] < str(end_temp)]

    for tick in np.unique(temp2["Ticker"]):
        ticker_temp = temp2[temp2["Ticker"] == tick]

        biweekly_data["start_date"].append(str(start_temp))
        biweekly_data["end_date"].append(str(end_temp))
        biweekly_data["ticker"].append(tick)

        all_lags = []
        for entry in ticker_temp.iterrows():
            all_lags.append(calculate_lag(entry[1]["Traded"], entry[1]["Published"]))
        biweekly_data["lag"].append(sum(all_lags) / len(all_lags))
        biweekly_data["n_trades"].append(len(ticker_temp))
        biweekly_data["Qty"].append(sum_quantity(ticker_temp["Qty"]))
        biweekly_data["Title"].append(find_unique_titles(ticker_temp["Title"]))

    start_temp = end_temp
    end_temp = start_temp + time_delta


pd.DataFrame(biweekly_data).to_csv("insiders_by_trade_date.biweekly.csv", index=False)

In [ ]:
# %% Save when sorting by declaration date

processed_data_DF_sorted = processed_data_DF.sort_values(by="Published")
processed_data_DF_sorted.to_csv("insiders_by_declared_date.csv", index=False)

start_temp = start_date
end_temp = start_date + time_delta
end_date = create_date(max([max(processed_data_DF_sorted["Traded"]), max(processed_data_DF_sorted["Published"])]))

biweekly_data = {}
biweekly_data["start_date"] = []
biweekly_data["end_date"] = []
biweekly_data["ticker"] = []
biweekly_data["lag"] = []
biweekly_data["n_trades"] = []
biweekly_data["Qty"] = []
biweekly_data["Title"] = []

while end_temp < end_date:

    temp1 = processed_data_DF_sorted[processed_data_DF_sorted["Published"] > str(start_temp)]
    temp2 = temp1[temp1["Published"] < str(end_temp)]

    for tick in np.unique(temp2["Ticker"]):
        ticker_temp = temp2[temp2["Ticker"] == tick]

        biweekly_data["start_date"].append(str(start_temp))
        biweekly_data["end_date"].append(str(end_temp))
        biweekly_data["ticker"].append(tick)

        all_lags = []
        for entry in ticker_temp.iterrows():
            all_lags.append(calculate_lag(entry[1]["Traded"], entry[1]["Published"]))
        biweekly_data["lag"].append(sum(all_lags) / len(all_lags))
        biweekly_data["n_trades"].append(len(ticker_temp))
        biweekly_data["Qty"].append(sum_quantity(ticker_temp["Qty"]))
        biweekly_data["Title"].append(find_unique_titles(ticker_temp["Title"]))

    start_temp = end_temp
    end_temp = start_temp + time_delta


pd.DataFrame(biweekly_data).to_csv("insiders_by_declared_date.biweekly.csv", index=False)